**Identificação do aluno**

**Email:**

**Matrícula:**

# Laboratório: DistilBERT com Máscara de Linguagem (IMDB)

Diferente dos laboratórios anteriores, onde a maioria das seções já estava pronta, **neste laboratório você será o protagonista da construção**.  
Você deverá se basear no exemplo visto em aula e **desenvolver sua própria versão**, colocando sua **personalidade** e aplicando **toques extras** para buscar um melhor desempenho.  

Relembrando o laboratório exemplificado em aula:  
- Ele utiliza o modelo **DistilBERT** em uma tarefa de **Masked Language Modeling (MLM)**.  
- O objetivo é treinar o modelo em um conjunto de dados (IMDB) e observar sua capacidade de prever palavras mascaradas em frases.  
- O processo envolve:  
  1. Preparação dos dados.  
  2. Tokenização.  
  3. Criação de máscaras nas frases.  
  4. Fine-tuning do modelo.  
  5. Avaliação dos resultados.  

Agora é a sua vez!  
Use o que foi mostrado em aula como **guia**, mas explore variações. Use essas **DICAS**:  
- Experimente **diferentes taxas de aprendizado**.  
- Aplique **diferentes tamanhos de batch**.  
- Teste **número de épocas distintos**.
- Use **diferentes taxas de mascaramento**.   
- Pense em **formas criativas de avaliar o desempenho**.  

O objetivo não é apenas replicar o exemplo, mas **melhorá-lo com suas próprias ideias**.

# Bibliotecas

In [ ]:
import tensorflow as tf
import numpy as np
from datasets import load_dataset
from transformers import DistilBertTokenizer, TFDistilBertForMaskedLM # DICA: Tente usar outras versões de BERT

# Conjunto de Dados

In [ ]:
# Esse conjunto de dados é relacionado ao IMDB (Plataforma que contempla resenhas sobre filmes)
dataset = load_dataset('imdb')
print(dataset['train'][0]['text'])

In [ ]:
from datasets import DatasetDict
from sklearn.model_selection import train_test_split

def stratified_subset_dataset(dataset_dict, n_samples_per_split, seed=42):
    """
    Recebe um DatasetDict (por exemplo, {'train': ..., 'test': ...})
    e retorna outro DatasetDict com a mesma estrutura,
    mas com no máximo n_samples_per_split exemplos em cada split,
    mantendo a proporção das labels.
    """
    new_splits = {}

    for split_name, split_data in dataset_dict.items():
        labels = np.array(split_data['label'])
        indices = np.arange(len(labels))
        # Seleção estratificada
        selected_indices, _ = train_test_split(
            indices,
            train_size=min(n_samples_per_split, len(labels)),
            stratify=labels,
            random_state=seed
        )
        new_splits[split_name] = split_data.select(selected_indices)

    return DatasetDict(new_splits)


subset_dataset = stratified_subset_dataset(dataset, n_samples_per_split=2000)  # DICA: tente modificar o parâmetro n_samples_per_split

In [ ]:
print(len(subset_dataset['train']))
print(len(subset_dataset['test']))

*   ## **Para as próximas etapas use o laborátorio apresentado em aula**

# Tokenização

Nessa seção vocês vão aprender a **transformar frases em tokens** (unidades menores de texto, como palavras ou subpalavras), que é o formato que o modelo consegue entender.  

Lembre-se:  
- O modelo **DistilBERT** não trabalha diretamente com texto cru, mas sim com **IDs numéricos** que representam tokens.  
- O **tokenizer** faz esse processo automaticamente:  
  1. Divide a frase em tokens.  
  2. Converte os tokens para IDs.  
  3. Garante que todos os exemplos tenham o mesmo tamanho (com *padding* e *truncation*). Lembre-se de tentar diferentes tamanhos de tokens de entrada do modelo.  

# Adicionando Mascara aos Dados

Nessa etapa vocês vão aplicar a **máscara de linguagem (MLM)**, que é o coração do pré-treinamento do BERT/DistilBERT.  
A ideia é **esconder (mascarar) aleatoriamente tokens** da frase e pedir para o modelo tentar prever quais palavras estavam lá.  

**Dica:**
- Não é preciso reinventar tudo: a biblioteca `transformers` já possui funções que ajudam a criar essas máscaras.  
- Testem mascarar as frases considerando outras proporções além dos 15% originais do artigo.   

# Adequando os Dados

Para treinar um modelo de linguagem de forma eficiente, precisamos **organizar os dados em lotes (batches)** e colocá-los em uma estrutura que o modelo consiga consumir durante o treinamento.  

**Dica:**
- Testem com diferentes tamanhos de `batch_size` e observem como isso afeta a memória e a velocidade.  
- Conferir os **shapes** dos tensores é sempre uma boa prática para evitar erros no treinamento.  

# Carregando dados de um modelo pré-treinado para continuar o pré-treinamento

Até aqui vocês aprenderam a preparar dados (tokenizar, mascarar e estruturar em batches).  
Agora, em vez de treinar um modelo do zero, vamos **aproveitar o conhecimento de um modelo já pré-treinado** e continuar o pré-treinamento com o nosso conjunto de dados (IMDB).  

Por que usar um modelo pré-treinado?
- Modelos como **DistilBERT** já foram treinados em enormes quantidades de texto.  
- Isso significa que eles **já entendem bastante da linguagem** (sintaxe, gramática, vocabulário, contexto).  
- Ao continuar o pré-treinamento com um novo corpus, você adapta o modelo para **captar melhor o estilo e o domínio** dos seus dados.  
- Lembre-se de experimentar diferentes quantidades de épocas e valores de learning-rate.  


# Testando as Predições

**Atenção use esses três exemplos para testar!!!**

In [ ]:
# Exemplo 1
text = "The movie was really [MASK]."

# Exemplo 2
text = "The food at the restaurant was absolutely [MASK]."

# Exemplo 3
text = "The weather today is very [MASK]."

Busque alguma frase de exemplo da base de dados de teste e coloque a máscara em lugares variados do texto.

In [ ]:
#Busque alguma frase no conjunto de teste
text = "... [MASK] ..."

# Finetuning Downstream (Classificação)

Até agora o foco foi no **pré-treinamento com Masked Language Modeling (MLM)**, onde o objetivo era prever palavras mascaradas.  
Mas o poder do BERT/DistilBERT aparece de verdade quando usamos esse conhecimento adquirido em **tarefas downstream** (tarefas específicas), como classificação.  

**O que muda aqui?**
- No pré-treinamento, o modelo aprende sobre a **linguagem em geral**.  
- No fine-tuning, ajustamos o modelo para uma tarefa **específica**, por exemplo:  
  - **Classificação de sentimentos** (positivo/negativo em reviews do IMDB).  
  - **Classificação de tópicos**.  
  - **Detecção de spam**.  

**O que vocês precisam fazer aqui:**
- Utilizar um modelo pré-treinado (pode ser o seu ou não) (`DistilBERT`).  
- Adaptá-lo para classificação usando `TFDistilBertForSequenceClassification` (ou versão PyTorch).  
- Preparar os dados de entrada (frases e rótulos → `0` para negativo, `1` para positivo, por exemplo).  
- Treinar o modelo nos dados rotulados.
  
**Objetivo final:**  
Transformar um modelo genérico (pré-treinado em linguagem) em um modelo **especializado em classificar sentimentos no IMDB**.

**Dicas:**
- Ajustem **taxa de aprendizado** e **batch size**: esses hiperparâmetros têm forte impacto no desempenho.  
- Depois de treinar, calcule as métricas para verificar se o modelo capturou bem os padrões de classificação.
- Tente obter resultados melhores do que os do notebook exemplo da aula.  

